### Build code to scrape Toronto postal codes from Wikipedia page

#### Import the libraries I will use.

In [1]:
from bs4 import BeautifulSoup

In [2]:
import lxml

In [3]:
import requests

In [4]:
import pandas as pd

#### Get the wikipedia article in text form using requests.get  Then create a file named soup using Beautiful Soup with that url as the source and lxlm as the parser.

In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [6]:
soup = BeautifulSoup(source, 'lxml')

#### Create an empty data file. Create a file named 'table' that has the table elements from the source file. Transfer the 'tr' features to rows.

In [7]:
data = []
table = soup.find('table')
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

#### Transfor the data into a pandas dataframe named new_table

In [8]:
new_table = pd.DataFrame(data)

#### Remove the empty first row, rename the columns, display the first rows of the dataframe

In [9]:
new_table2=new_table.loc[1:len(new_table), ]
new_table2 = new_table2.rename(columns = {0:'PostalCode',1:'Borough',2:'Neighborhood'})
new_table2.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


#### If any rows have a Borough assigned but not a Neighborhood, assign the Borough name to the Neighborhood name

In [10]:
for row in new_table2.itertuples():
    if(row.Neighborhood=="Not assigned"):
        if(row.Borough!="Not assigned"):
            rows=(row.Index)
            new_table2.at[rows, 'Neighborhood'] = new_table2.at[rows, 'Borough']
     

#### Check that everything looks good so far

In [11]:
new_table2.head(10)

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
10,M8A,Not assigned,Not assigned


#### Remove rows where the Neighborhood value is 'not assigned'

In [12]:
new_table2 = new_table2[new_table2['Neighborhood'] != "Not assigned"]
new_table2.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


#### Aggregate rows so that neighborhoods are combined in a single row if they have the same postal code and Borough

In [13]:
new_table3 = new_table2.groupby(by=['PostalCode', 'Borough'], sort=False).agg( ','.join).reset_index()

new_table3.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [14]:
new_table3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [15]:
latlon = pd.read_csv('/resources/data/Geospatial_data.csv')
latlon.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [16]:
latlon.columns=['PostalCode','Latitude','Longitude']
latlon.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
finalt = pd.merge(new_table3, latlon, on='PostalCode', how='outer') #merge the latitude-longitude table with the Borough table

finalt.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [20]:
finalt.info()   #Make sure the final data frame is correct

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
Latitude        103 non-null float64
Longitude       103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [24]:
finalt.to_csv('data\TorontoBo.csv') #write file to a csv to use again